In [1]:
#from keras.models import Sequential
#from keras.layers import Conv2D , MaxPooling2D

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

Using TensorFlow backend.


In [4]:
train_data = ImageDataGenerator(rescale = 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_data = ImageDataGenerator(rescale=1./25)
train = train_data.flow_from_directory('C:/Users/ASA/Desktop/training_data',target_size=(64,64),batch_size = 32,class_mode='categorical')
test = test_data.flow_from_directory('C:/Users/ASA/Desktop/test',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 4540 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [5]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
#Simple K-Fold cross validation. 10 folds.
cv = cross_validation.KFold(len(train), n_folds=10)

C:\Users\ASA\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units= 128 , activation = 'relu'))
model.add(Dense(units=4,activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
__________

In [7]:
model.compile(optimizer = 'adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [9]:
model.fit_generator(train,steps_per_epoch=3000,epochs = 6,validation_data=test,validation_steps=1500)

Epoch 1/6
3000/3000 [==============================] - 1740s 580ms/step - loss: 0.0051 - acc: 0.9985 - val_loss: 0.8193 - val_acc: 0.9357
Epoch 2/6
3000/3000 [==============================] - 1689s 563ms/step - loss: 0.0032 - acc: 0.9991 - val_loss: 0.6792 - val_acc: 0.9534
Epoch 3/6
3000/3000 [==============================] - 1708s 569ms/step - loss: 0.0029 - acc: 0.9993 - val_loss: 1.0095 - val_acc: 0.9251
Epoch 4/6
3000/3000 [==============================] - 1760s 587ms/step - loss: 0.0021 - acc: 0.9995 - val_loss: 0.4110 - val_acc: 0.9701
Epoch 5/6
3000/3000 [==============================] - 1763s 588ms/step - loss: 1.5423e-04 - acc: 1.0000 - val_loss: 0.8880 - val_acc: 0.9392
Epoch 6/6
3000/3000 [==============================] - 1860s 620ms/step - loss: 1.3253e-06 - acc: 1.0000 - val_loss: 1.0224 - val_acc: 0.9328


In [11]:
train.class_indices

{'sitting_corect_v2': 0,
 'sitting_incorrect_v2': 1,
 'standing_corect_v2': 2,
 'standing_incorrect_v2': 3}

In [67]:
from keras.preprocessing import image
imagetest = image.load_img('C:/Users/ASA/Desktop/8.jpg',target_size=(64,64))

In [68]:
imagetest = image.img_to_array(imagetest)

In [69]:
imagetest = np.expand_dims(imagetest,axis = 0)

In [70]:
imagetest

array([[[[  84.,   42.,   28.],
         [  75.,   39.,   27.],
         [  96.,   58.,   39.],
         ..., 
         [ 189.,  194.,  198.],
         [ 191.,  194.,  199.],
         [ 187.,  192.,  196.]],

        [[  81.,   44.,   28.],
         [  77.,   41.,   27.],
         [  84.,   46.,   27.],
         ..., 
         [ 188.,  191.,  198.],
         [ 190.,  193.,  198.],
         [ 188.,  191.,  198.]],

        [[  83.,   46.,   30.],
         [  82.,   45.,   36.],
         [  91.,   54.,   38.],
         ..., 
         [ 184.,  187.,  194.],
         [ 189.,  192.,  199.],
         [ 191.,  194.,  201.]],

        ..., 
        [[ 181.,  180.,  178.],
         [ 180.,  180.,  178.],
         [ 181.,  180.,  178.],
         ..., 
         [ 208.,  208.,  208.],
         [ 209.,  209.,  209.],
         [ 204.,  204.,  204.]],

        [[ 180.,  179.,  177.],
         [ 179.,  178.,  176.],
         [ 182.,  180.,  181.],
         ..., 
         [ 207.,  207.,  207.],
       

In [71]:
result=model.predict(imagetest)

In [72]:
result

array([[ 1.,  0.,  0.,  0.]], dtype=float32)

In [63]:
if result[0][0]==1:
    prediction = 'sitting correct'
if result[0][0]==2:
    prediction = 'sitting incorrect'
if result[0][0]==3:
    prediction = 'standing correct'
if result[0][0]==4:
    prediction = 'standing incorrect'